In [1]:
# ! pip install pandarallel
# from pandarallel import pandarallel
import json
import re
import pandas as pd
import numpy as np
import itertools
from numba import jit, njit

In [2]:
# import multiprocessing

# num_cores = multiprocessing.cpu_count()

# print(f"The number of cores in your system is: {num_cores}")

In [3]:
df = pd.read_csv('Data1.csv')

# remove blank spaces from front and back of column names
df.columns = df.columns.str.strip()
df.columns

<ipython-input-3-fc8b3e05b5c4>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data1.csv')


Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG'],
      dtype='object')

In [4]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example1.RSE', "r") as f:
    print(type(f))
    for line in f:
        print(line)
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

<class '_io.TextIOWrapper'>
'Example1.RSE – file will be as follows:

#SAMPLING SUBSAMPLE2	'SUBSAMPLE2 Key word will be used for this scheme

#GROUP 2			'Number of Group variables used

SEC   C sec 	'Sector Rural-1, Urban-2

ST_GR C NEWVAR 	'Groups of States



#FILTER 1		'Number of conditions used to generate the Work-file

AGE>=15



#VARIABLE 3  	'Number of Variables selected

POP	N  NEWVAR	'Variable for Population

LF	N  NEWVAR	'Variable for Labour Force

WRK	N  NEWVAR	'Variable for Workers



#FILE Data1.csv  	'We may make it optional, directly select the file in the app



#RENAME  5    			'Number of variables to be renamed

STRMID C sec+st+strm+sstrm    'Combination of fields to get unique Stratum ID

SS     C ss				'Sub Sample Code

NSS    N  nss			'No. of FSU surveyed in STRM_ID X SS

NSC    N  nsc			'No. of FSU surveyed in STRM_ID

MULT   N  mult			'Sub-sample Multiplier X 100



#TRANSFORM 4  	' Number of Variable transformed

ST_GR=("G1")   ST in ("10,19,20,21")			'For East

In [5]:
def SAMPLING_(data, df):
    for key, value in data.items():
        ## To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')

            elif str('SRSWR').lower() in str(key).lower():
                pass
            elif str('SRSWOR').lower() in str(key).lower():
                pass

SAMPLING_(data, df)

'SUBSAMPLE2'

In [6]:
### ---------------------------------------------------------------------------------------------------------
def GROUP_(data, df):
    for key, value in data.items():
        if str('GROUP').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            for name in value:
                if name in df.columns:
                    pass
                elif name not in df.columns:
                    df[name] = 0

            return value


    for key, value in data.items():
        print(key, value)
        ### To read the group columns
        if str('GROUP').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\w+\s+\w+\s+\w+'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst = match.group().split()

                print(lst[0], lst[1])

                if lst[0] in df.columns:
                    if lst[1] == 'C':
                        df[lst[0]] = df[lst[0]].astype(str)
                elif lst[0] not in df.columns:
                    # if lst[1] == 'C':
                    #     df[lst[0]] = ''
                    pass

        else:
            pass

### ---------------------------------------------------------------------------------------------------------
def FILTER_(data, df):
    lst_queries = []
    for key, value in data.items():

        if str('FILTER').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\([^()]+\)|\w+[<>!=]+[0-9]+(?:\s*(?:&&|\|\|)\s*(?:\([^()]+\)|\w+[<>!=]+[0-9]+))*'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst_queries.append(match.group())

        else:
            pass

    filter_query = ' & '.join(['({})'.format(item) for item in lst_queries])

    try:
        df = df.query(filter_query)
        return df
    except:
        return df


### ---------------------------------------------------------------------------------------------------------
def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

### ---------------------------------------------------------------------------------------------------------
def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if int(i) < 10 else str(i) for i in x]), axis=1)


                else:
                    col_name = col_name.upper()

### ---------------------------------------------------------------------------------------------------------
def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]

            for name in value:
                df[name] = 1

### ---------------------------------------------------------------------------------------------------------
def TRANSFORM_(data):
    global condition

    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                print(line_values)
                var_name = line_values[0].strip()  # extract the variable name

                var_value = line_values[1].strip().split()[0]  # extract the variable value


                # Regular expression pattern to extract the condition
                pattern = r'^\w+=\([^\)]*\)(?:\s+(.*?)(?:\n|$))?$'

                # Split the string into lines and extract the condition from each line
                conditions = []

                for line in line.split('\n'):
                    match = re.match(pattern, line)

                    if match:
                        condition = match.group(1)

                        if condition is not None:
                            conditions.append(condition.strip())
                        else:
                            conditions.append(None)

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)

                Arr3.append(condition)

    return Arr1, Arr2, Arr3

### ---------------------------------------------------------------------------------------------------------
def EST_RSE_(data):
    for key, value in data.items():
        pass

df = FILTER_(data, df)
GROUP_(data, df)
TRANSFORM_(data)
RENAME_(data, df)
NEWVARIABLE_(data, df)
FILE_(data)



<ipython-input-6-16660ca9d900>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name] = 0


['ST_GR', '("G1")   ST in ("10,19,20,21")']
['ST_GR', '("G2")   ST in ("11,12,13,14,15,16,17,18")']
['ST_GR', '("G3")   Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)      CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)      Otherwise']


'Data1.csv'

In [7]:
def parse_query_string(query_string):
    # Replace curly quotes with straight quotes
    query_string = query_string.replace("”", "\"")

    if 'in' in query_string:
        if ('NOT in' or 'not in') in query_string:
            query_string = re.sub(r'(\b\w+\b)\s+NOT in\s*\(([^)]+)\)', r'~\1.isin([\2])', query_string)
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        elif 'in' in query_string:
            # Replace 'in' operators with 'isin' and add list brackets
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        else:
            pass

    else:
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)('[^']*'|"[^"]*")|(\b\w+\b)(\s*[<>=!]+\s*)("[^]*'|"[^"]*")''', r'\1\2\3', query_string)
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)(\d+)''', r'\1\2\3', query_string)

    # Replace double quotes with single quotes
    query_string = query_string.replace("\"", "'")

    # Replace 'AND' with '&', 'OR' with '|', and add parentheses
    query_string = query_string.replace(" AND ", " & ")\
                        .replace(" OR ", " | ").\
                            replace("&&", '&').\
                                replace('||', '|').\
                                    replace('<>', '!=').\
                                        replace('NOT', '~')

    # Replace column names with df[column] syntax
    query_string = re.sub(r'(\b\w+\b)=', r'\1==', query_string)


    # # Define a function to replace variable names with df["<variable_name>"]
    # def replace_var(match):
    #     return f'df["{match.group(0)}"]'

    # # Use regex to replace variable names with df["<variable_name>"]
    # pattern = re.compile(r'''\b\w+\b(?=\s*[<>=!]=*\s*[\'"\d])''')
    # new_string = pattern.sub(replace_var, query_string)

    query_string = re.sub(r"'(\d+)'|\"(\d+)\"", lambda match: str(int(match.group(1) or match.group(2))), query_string)
    query_string = re.sub(r'\[(\d+,\s*)+\d+\]', lambda match: '[' + match.group(0).replace("\'", "'") + ']', query_string)

    return query_string


final_parsed_query = []

for i in list(TRANSFORM_(data))[2]:
    # print(i)
    if i:
        parsed_query = parse_query_string(i)
        final_parsed_query.append(parsed_query)
    else:
        final_parsed_query.append(i)

### Calling the final_parsed_query function
### ---------------------------------------
final_parsed_query

['ST_GR', '("G1")   ST in ("10,19,20,21")']
['ST_GR', '("G2")   ST in ("11,12,13,14,15,16,17,18")']
['ST_GR', '("G3")   Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)      CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)      Otherwise']


["ST.isin(['10,19,20,21'])",
 "ST.isin(['11,12,13,14,15,16,17,18'])",
 'Otherwise',
 None,
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])",
 'Otherwise']

In [8]:
Arr1 = list(TRANSFORM_(data))[0]
Arr2 = list(TRANSFORM_(data))[1]

# print(Arr2)
Arr2 = [i.strip('()\'\"') for i in Arr2]
Arr3 = final_parsed_query

Arr2 = [float(elem) if elem.replace('.', '', 1).isdigit() else elem for elem in Arr2]
print(Arr2)

# print(Arr1)
# print(Arr2)
# print(Arr3)

['ST_GR', '("G1")   ST in ("10,19,20,21")']
['ST_GR', '("G2")   ST in ("11,12,13,14,15,16,17,18")']
['ST_GR', '("G3")   Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)      CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)      Otherwise']
['ST_GR', '("G1")   ST in ("10,19,20,21")']
['ST_GR', '("G2")   ST in ("11,12,13,14,15,16,17,18")']
['ST_GR', '("G3")   Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)      CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)      Otherwise']
['G1', 'G2', 'G3', 1.0, 1.0, 0.0, 1.0, 0.0]


In [9]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

set()

In [10]:
Arr3 = [i.replace("(['", "([").replace("'])", "])") if i is not None else 'True' if Arr3[j] == 'Otherwise' else True for j, i in enumerate(Arr3)]
Arr3

['ST.isin([10,19,20,21])',
 'ST.isin([11,12,13,14,15,16,17,18])',
 'Otherwise',
 True,
 'CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 'CWS.isin([11,12,21,31,41,51,61,61,71,72,81])',
 'Otherwise']

In [11]:
new_arr3 = []
for i, value in enumerate(Arr3):
    if value == 'Otherwise':
        other_values = set(['~(' + Arr3[j] + ')' for j in range(len(Arr3)) if Arr1[j] == Arr1[i] and Arr3[j] != 'Otherwise'])
        new_value = ' & '.join(other_values)
        print(new_value)
        new_arr3.append(new_value)
    else:
        new_arr3.append(value)
        print(value)

# new_arr3

ST.isin([10,19,20,21])
ST.isin([11,12,13,14,15,16,17,18])
~(ST.isin([11,12,13,14,15,16,17,18])) & ~(ST.isin([10,19,20,21]))
True
CWS.isin([11,12,21,31,41,51,61,61,71,72])
~CWS.isin([11,12,21,31,41,51,61,61,71,72])
CWS.isin([11,12,21,31,41,51,61,61,71,72,81])
~(CWS.isin([11,12,21,31,41,51,61,61,71,72,81]))


In [12]:
Arr3 = new_arr3

In [13]:
for i in range(len(Arr2)):
    if type(Arr3[i]) == str:
        print(Arr3[i])

ST.isin([10,19,20,21])
ST.isin([11,12,13,14,15,16,17,18])
~(ST.isin([11,12,13,14,15,16,17,18])) & ~(ST.isin([10,19,20,21]))
CWS.isin([11,12,21,31,41,51,61,61,71,72])
~CWS.isin([11,12,21,31,41,51,61,61,71,72])
CWS.isin([11,12,21,31,41,51,61,61,71,72,81])
~(CWS.isin([11,12,21,31,41,51,61,61,71,72,81]))


In [14]:
# Apply the queries in Arr3 to filter the rows of the dataframe
for i, query_str in enumerate(Arr3):

    if query_str is True:
        if type(Arr2[i]) == str:
            if '+' in Arr2[i]:
                df[Arr1[i]] = df[Arr2[i].split('+')].apply(lambda x: x.sum(), axis=1)
                print('*** First if', Arr2[i], Arr1[i], Arr3[i], i)

            else:
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                print('*** First else', Arr2[i], Arr1[i], Arr3[i], i)
        # if type(Arr3[i]):
        #     print('^^^ Second if', Arr2[i], Arr1[i], Arr3[i], i)
        #     df.loc[df_query.index, Arr1[i]] = Arr2[i]

    else:
        # try:
        # query_str = re.sub(r'0?\s?(\d+)', r'\1', query_str)
        print(query_str)
        df_query = df.query(query_str)
        # except:
        #     pass

        if type(Arr2[i]) == float:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('### Third if', Arr2[i], Arr1[i], Arr3[i])
        else:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('@@@ First else', Arr2[i], Arr1[i], Arr3[i])


        if type(Arr2[i]) == str:
            if Arr2[i] in list(df.columns):
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                # print('$$$ Fourth if', Arr2[i], Arr1[i], Arr3[i])

# print(df)

ST.isin([10,19,20,21])
ST.isin([11,12,13,14,15,16,17,18])
~(ST.isin([11,12,13,14,15,16,17,18])) & ~(ST.isin([10,19,20,21]))
CWS.isin([11,12,21,31,41,51,61,61,71,72])
~CWS.isin([11,12,21,31,41,51,61,61,71,72])
CWS.isin([11,12,21,31,41,51,61,61,71,72,81])
~(CWS.isin([11,12,21,31,41,51,61,61,71,72,81]))


In [15]:
df['MULT'] = pd.to_numeric(df['MULT'])

# Calculate the SS_MULT and CMULT fields
df['SS_MULT'] = df['MULT'] / 100
df['CMULT'] = df.apply(lambda row: row['MULT'] / 200 if row['NSC'] > row['NSS'] else row['MULT'] / 100, axis=1)


In [16]:
df.to_csv('WorkFile_Balanced_Half_Sample_Method_Example1.csv')

In [17]:
bin(8)

'0b1000'

In [18]:
def generate_binary_combinations(n):
    binary_combinations = []
    for i in range(2 ** n):
        binary_combination = bin(i)[2:]
        binary_combination = '0' * (n - len(binary_combination)) + binary_combination
        binary_combinations.append(binary_combination)
    return binary_combinations

# limit = int(input(2))
limit = 2
binary_of_no = generate_binary_combinations(limit)
print(binary_of_no)

['00', '01', '10', '11']


In [19]:
def generate_variable_mapping(GROUP, binary_combinations):
    mapping = {}

    for comb in binary_combinations:
        variable_values = []
        consecutive_zeros = 0

        for i, bit in enumerate(comb):
            if bit == '0':
                consecutive_zeros += 1
                variable_values.append('Z' * consecutive_zeros)


            else:
                consecutive_zeros = 0  # Reset the consecutive_zeros count
                variable_values.append(GROUP[i])
        mapping[comb] = variable_values

    return mapping

###
GROUP = GROUP_(data, df)

# Create an empty DataFrame
df_new = pd.DataFrame()

n = len(GROUP)
binary_combinations = generate_binary_combinations(n)
variable_mapping = generate_variable_mapping(GROUP, binary_combinations)

VALUES = []
print("Customized Mapping:")
for comb, values in variable_mapping.items():
    print(f"Binary Combination '{comb}': {values}")
    VALUES.append(values)

Customized Mapping:
Binary Combination '00': ['Z', 'ZZ']
Binary Combination '01': ['Z', 'ST_GR']
Binary Combination '10': ['SEC', 'Z']
Binary Combination '11': ['SEC', 'ST_GR']


In [20]:
VALUES

[['Z', 'ZZ'], ['Z', 'ST_GR'], ['SEC', 'Z'], ['SEC', 'ST_GR']]

In [21]:
# import pandas as pd

# # # Sample DataFrame
# # data = {'A': [1, 2, 3],
# #         'B': [4, 5, 6],
# #         'ST_GR': [1, 2, 3],
# #         'SEC': [1, 2, 4]}
# # df = pd.DataFrame(data)

# GROUP = ['SEC', 'ST_GR']
# VALUES = [['Z', 'ZZ'], ['Z', 'ST_GR'], ['SEC', 'Z'], ['SEC', 'ST_GR']]

# ### Storing group variables into temporary dataframe
# temp_df = df[GROUP].copy()

# # Create a dictionary to store the updates
# updates = {}

# ### Iterate through GROUP and VALUES
# for value in range(len(VALUES)):
#     for i in range(len(GROUP)):
#         if GROUP[i] != VALUES[value][i]:
#             updates[GROUP[i]] = VALUES[value][i]
#             print(VALUES[value][i])

#         if GROUP[i] == VALUES[value][i]:
#             updates[GROUP[i]] = temp_df[GROUP[i]]

#     # Update the DataFrame with the specified values
#     df.update(pd.DataFrame(updates, index=df.index))

#     print("Updated DataFrame:")
#     print(df)


In [22]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'ST_GR',
       'STRMID', 'POP', 'LF', 'WRK', 'SS_MULT', 'CMULT'],
      dtype='object')

In [23]:
### Storing group variables into temporary dataframe
temp_df = df[GROUP].copy()

# Create a dictionary to store the updates
updates = {}

Tables = []

### Iterate through GROUP and VALUES
for value in range(len(VALUES)):
    for i in range(len(GROUP)):
        if GROUP[i] != VALUES[value][i]:
            updates[GROUP[i]] = VALUES[value][i]
#             print(VALUES[value][i])

        if GROUP[i] == VALUES[value][i]:
            updates[GROUP[i]] = temp_df[GROUP[i]]

    # Update the DataFrame with the specified values
    df.update(pd.DataFrame(updates, index=df.index))

    # GROUP = ['SEC', 'ST_GR']
    new_groupby_cols_subset = ['STRMID']

    ### Step 1: Generate Group level estimates Yhat & Xhat
    w1 = df.groupby(GROUP).agg(
                        no_sam=('POP', 'count'),
                        pophat=('POP', lambda x: np.sum(x * df['CMULT'])),
                        lfhat=('LF', lambda x: np.sum(x * df['CMULT'])),
                        wrkhat=('WRK', lambda x: np.sum(x * df['CMULT']))
                    ).reset_index()


    @njit
    def calculate_stratum_estimates(pop, ss, ss_mult, cmult, lf, wrk):
        popstr = np.sum(pop * cmult)
        s1pop = np.nansum(np.where(ss == 1, pop * ss_mult, 0.0))
        s2pop = np.nansum(np.where(ss == 2, pop * ss_mult, 0.0))
        lfstr = np.nansum(lf * cmult)
        s1lf = np.nansum(np.where(ss == 1, lf * ss_mult, 0.0))
        s2lf = np.nansum(np.where(ss == 2, lf * ss_mult, 0.0))
        wrkstr = np.nansum(wrk * cmult)
        s1wrk = np.nansum(np.where(ss == 1, wrk * ss_mult, 0.0))
        s2wrk = np.nansum(np.where(ss == 2, wrk * ss_mult, 0.0))

        return popstr, s1pop, s2pop, lfstr, s1lf, s2lf, wrkstr, s1wrk, s2wrk

    # Apply the Numba-compiled function to the DataFrame
    w2 = df.groupby(GROUP + new_groupby_cols_subset).apply(
        lambda x: pd.Series(calculate_stratum_estimates(x['POP'].values, x['SS'].values, x['SS_MULT'].values,
                                                        x['CMULT'].values, x['LF'].values, x['WRK'].values),
                        index=['popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf', 'wrkstr', 's1wrk', 's2wrk'])
    ).reset_index()



    # Step 4: Combine all estimates
    w3A = pd.merge(w2, w1, on=GROUP)

    # Include R1hat and R2hat
    w3A['R1hat'] = w3A['lfhat'] / w3A['pophat']
    w3A['R2hat'] = w3A['wrkhat'] / w3A['pophat']

    # Step 5: Equating Stratum & Sub-Sample estimates
    w3A['s1pop'] = np.where(w3A['popstr'] == w3A['s2pop'], w3A['popstr'], w3A['s1pop'])
    w3A['s2pop'] = np.where(w3A['popstr'] == w3A['s1pop'], w3A['popstr'], w3A['s2pop'])
    w3A['s1lf'] = np.where(w3A['lfstr'] == w3A['s2lf'], w3A['lfstr'], w3A['s1lf'])
    w3A['s2lf'] = np.where(w3A['lfstr'] == w3A['s1lf'], w3A['lfstr'], w3A['s2lf'])
    w3A['s1wrk'] = np.where(w3A['wrkstr'] == w3A['s2wrk'], w3A['wrkstr'], w3A['s1wrk'])
    w3A['s2wrk'] = np.where(w3A['wrkstr'] == w3A['s1wrk'], w3A['wrkstr'], w3A['s2wrk'])


    @njit
    def compute_var_and_mse_np(s1pop, s2pop, s1lf, s2lf, R1hat, s1wrk, s2wrk, R2hat):
        var_pop = np.sum((s1pop - s2pop) ** 2)

        mse_R1 = np.nansum(
            ((s1lf - s2lf) ** 2) +
            ((R1hat ** 2) * ((s1pop - s2pop) ** 2) -
            (2 * R1hat * (s1lf - s2lf) * (s1pop - s2pop)))
        )

        mse_R2 = np.nansum(
            ((s1wrk - s2wrk) ** 2) +
            ((R2hat ** 2) * ((s1pop - s2pop) ** 2) -
            (2 * R2hat * (s1wrk - s2wrk) * (s1pop - s2pop)))
        )

        return var_pop, mse_R1, mse_R2

    # Use the function in your groupby
    w4 = w3A.groupby(GROUP + ['no_sam', 'pophat', 'lfhat', 'wrkhat', 'R1hat', 'R2hat']).apply(
        lambda x: pd.Series(compute_var_and_mse_np(x['s1pop'].values, x['s2pop'].values, x['s1lf'].values,
                                                    x['s2lf'].values, x['R1hat'].values, x['s1wrk'].values,
                                                    x['s2wrk'].values, x['R2hat'].values),
                        index=['var_pop', 'mse_R1', 'mse_R2'])
    ).reset_index()

    # print(w4)

    w5 = w4.copy()
    w5['RSE_POP'] = 100 * np.sqrt(w4['var_pop']) / (2 * w4['pophat'])
    w5['RSE_R1'] = 100 * np.sqrt(w4['mse_R1']) / (2 * w4['lfhat'])
    w5['RSE_R2'] = 100 * np.sqrt(w4['mse_R2']) / (2 * w4['wrkhat'])

    w5['R1'] = 100 * w5['R1hat']
    w5['R2'] = 100 * w5['R2hat']

    # Step 9: Rearrange the fields
    T1 = w5[GROUP + ['no_sam', 'pophat', 'RSE_POP', 'R1', 'RSE_R1', 'R2', 'RSE_R2']]

    Tables.append(T1)


### Merging all tables
### ------------------
def merge_dataframes(tables):
    merged_df = tables[0]
    for df in tables[1:]:
        merged_df = pd.concat([merged_df, df], axis=0)
    return merged_df


df_Tables = merge_dataframes(Tables)

df_Tables


,SEC,ST_GR,no_sam,pophat,RSE_POP,R1,RSE_R1,R2,RSE_R2
0,Z,ZZ,83801,8.050829e+08,0.709544,47.524631,0.639070,44.047162,0.699980
0,Z,G1,14365,1.769427e+08,1.595048,42.847025,1.539794,39.846577,1.625851
1,Z,G2,11786,2.935630e+07,2.773129,48.885000,1.833960,44.738082,2.109724
2,Z,G3,57650,5.987839e+08,0.818214,48.840186,0.724708,45.254576,0.802216
0,1,Z,46719,5.591594e+08,0.773705,48.276147,0.833311,44.950766,0.913596
1,2,Z,37082,2.459235e+08,1.516864,45.815900,0.856469,41.992625,0.923299
0,1,G1,9158,1.441490e+08,1.827488,42.854780,1.834435,39.943801,1.916883
1,1,G2,7202,2.483506e+07,3.122312,49.414335,2.106586,45.340498,2.425839
2,1,G3,30359,3.901753e+08,0.856789,50.206607,0.973324,46.775767,1.085238
3,2,G1,5207,3.279368e+07,3.088637,42.812940,1.998743,39.419213,2.412273


In [24]:
w3A

,SEC,ST_GR,STRMID,popstr,s1pop,s2pop,lfstr,s1lf,s2lf,wrkstr,s1wrk,s2wrk,no_sam,pophat,lfhat,wrkhat,R1hat,R2hat
0,1,G1,01100101,1743248.160,1786450.69,1700045.63,690054.870,822334.45,557775.29,690054.870,822334.45,557775.29,9158,1.441490e+08,6.177476e+07,5.757861e+07,0.428548,0.399438
1,1,G1,01100102,1001093.075,1045280.21,956905.94,363007.660,409913.80,316101.52,350482.515,384863.51,316101.52,9158,1.441490e+08,6.177476e+07,5.757861e+07,0.428548,0.399438
2,1,G1,01100103,1085484.785,1308408.33,862561.24,384973.330,340467.54,429479.12,384973.330,340467.54,429479.12,9158,1.441490e+08,6.177476e+07,5.757861e+07,0.428548,0.399438
3,1,G1,01100104,1321882.215,1403425.71,1240338.72,473290.850,547558.01,399023.69,473290.850,547558.01,399023.69,9158,1.441490e+08,6.177476e+07,5.757861e+07,0.428548,0.399438
4,1,G1,01100105,1240594.895,1362166.16,1119023.63,491118.940,596017.19,386220.69,491118.940,596017.19,386220.69,9158,1.441490e+08,6.177476e+07,5.757861e+07,0.428548,0.399438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,2,G3,02360103,169923.040,241404.23,98441.85,96043.930,150505.11,41582.75,71165.075,104683.01,37647.14,27291,2.086086e+08,9.655337e+07,8.846961e+07,0.462845,0.424094
1110,2,G3,02360104,5671978.540,5504905.74,5839051.34,2836082.760,2761090.07,2911075.45,2659533.145,2503083.04,2815983.25,27291,2.086086e+08,9.655337e+07,8.846961e+07,0.462845,0.424094
1111,2,G3,02360201,424206.270,353496.22,494916.32,257410.475,174678.46,340142.49,243912.755,174678.46,313147.05,27291,2.086086e+08,9.655337e+07,8.846961e+07,0.462845,0.424094
1112,2,G3,02360202,1175570.870,1064958.25,1286183.49,634584.645,577475.84,691693.45,586241.900,561536.04,610947.76,27291,2.086086e+08,9.655337e+07,8.846961e+07,0.462845,0.424094


In [25]:
df_Tables.to_csv('Balanced_Half_Sample_Final_Table.csv')

In [26]:
df_Tables

,SEC,ST_GR,no_sam,pophat,RSE_POP,R1,RSE_R1,R2,RSE_R2
0,Z,ZZ,83801,8.050829e+08,0.709544,47.524631,0.639070,44.047162,0.699980
0,Z,G1,14365,1.769427e+08,1.595048,42.847025,1.539794,39.846577,1.625851
1,Z,G2,11786,2.935630e+07,2.773129,48.885000,1.833960,44.738082,2.109724
2,Z,G3,57650,5.987839e+08,0.818214,48.840186,0.724708,45.254576,0.802216
0,1,Z,46719,5.591594e+08,0.773705,48.276147,0.833311,44.950766,0.913596
1,2,Z,37082,2.459235e+08,1.516864,45.815900,0.856469,41.992625,0.923299
0,1,G1,9158,1.441490e+08,1.827488,42.854780,1.834435,39.943801,1.916883
1,1,G2,7202,2.483506e+07,3.122312,49.414335,2.106586,45.340498,2.425839
2,1,G3,30359,3.901753e+08,0.856789,50.206607,0.973324,46.775767,1.085238
3,2,G1,5207,3.279368e+07,3.088637,42.812940,1.998743,39.419213,2.412273
